In [20]:
import os
import pandas as pd
import optuna
import numpy as np
import openml
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import kaleido
import plotly
import scipy.stats as stats
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from optuna.samplers import TPESampler, CmaEsSampler
from xgboost import XGBClassifier
from plotly.io import show
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.metrics import roc_auc_score

In [21]:
dataset_ids = [
    1590,   # Adult
    1510,   # Breast Cancer Wisconsin (Diagnostic)
    1461,   # Bank Marketing
    24,     # Mushroom
    40945   # Titanic
]

In [22]:
def fetch_and_prepare(openml_id):
    global y
    global X
    dataset = openml.datasets.get_dataset(openml_id)
    print(f">>> {dataset.name} (ID: {openml_id})")
    X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute, dataset_format='dataframe')
    for col in X.select_dtypes(include=['category', 'object']):
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
    imputer = SimpleImputer(strategy='mean')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    #scaler = MinMaxScaler()
    #X_imputed = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X.columns)
    if y.dtype == 'category':
        y = LabelEncoder().fit_transform(y)
    zbior = X_imputed['Target'] = y
    return X_imputed

In [23]:
zbior = fetch_and_prepare(1590)

>>> adult (ID: 1590)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(zbior.iloc[:, :-1], zbior['Target'], test_size=0.30, random_state=42)

def objective(trial):

    n_estimators = trial.suggest_int("n_estimators", 206, 1740)                    
    bootstrap = trial.suggest_categorical("bootstrap", ["True", "False"])
    max_features = trial.suggest_float("max_features", 0.323, 0.974)
    min_samples_split = trial.suggest_float("min_samples_split", 0.007, 0.513)

    bootstrap = bootstrap == "True"

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    wyniki = []

    for train_index, test_index in kf.split(X_train):
        X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

        X_train_fold = X_train_fold.reset_index(drop=True)
        X_test_fold = X_test_fold.reset_index(drop=True)
        y_train_fold = y_train_fold.reset_index(drop=True)
        y_test_fold = y_test_fold.reset_index(drop=True)

        X_train_fold = list(zip(*[X_train_fold[col] for col in X_train_fold]))
        X_test_fold = list(zip(*[X_test_fold[col] for col in X_test_fold]))

        Las_losowy = RandomForestClassifier(n_estimators = n_estimators, bootstrap = bootstrap, min_samples_split=min_samples_split,
                                            max_features = max_features)

        model = Las_losowy.fit(X_train_fold, y_train_fold)


        y_pred_proba = model.predict_proba(X_test_fold)[:, 1]
        auc = roc_auc_score(y_test_fold, y_pred_proba)
                                            
        wyniki.append(auc)

    rezultat = np.mean(wyniki)
    return rezultat

In [26]:
study = optuna.create_study(direction='maximize',sampler=TPESampler())
wynik = study.optimize(objective, n_trials=10)

[I 2025-11-12 21:44:32,123] A new study created in memory with name: no-name-f558a0eb-82fc-4d94-b8ba-d5716ba89034
[I 2025-11-12 21:45:58,876] Trial 0 finished with value: 0.8807491384042982 and parameters: {'n_estimators': 1604, 'bootstrap': 'False', 'max_features': 0.404202900659685, 'min_samples_split': 0.48354292463567133}. Best is trial 0 with value: 0.8807491384042982.
[I 2025-11-12 21:47:10,536] Trial 1 finished with value: 0.884579609569575 and parameters: {'n_estimators': 454, 'bootstrap': 'False', 'max_features': 0.7579631374346929, 'min_samples_split': 0.32350853902474924}. Best is trial 1 with value: 0.884579609569575.
[I 2025-11-12 21:48:17,855] Trial 2 finished with value: 0.8778171340998098 and parameters: {'n_estimators': 359, 'bootstrap': 'False', 'max_features': 0.9221085849980182, 'min_samples_split': 0.3182783628801951}. Best is trial 1 with value: 0.884579609569575.
[I 2025-11-12 21:50:42,317] Trial 3 finished with value: 0.9029763603430017 and parameters: {'n_estim

In [27]:
study.best_params

{'n_estimators': 555,
 'bootstrap': 'False',
 'max_features': 0.6767722944967449,
 'min_samples_split': 0.06123690495575964}

In [28]:
study.best_value

0.9029763603430017

In [29]:
#Defaultowe
Las_losowy = RandomForestClassifier(n_estimators = 500, bootstrap = bool("True"), min_samples_split= 1.0,
                                            max_features =  1.0)
model = Las_losowy.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(auc)

0.5


In [30]:
#Optymalne
Las_losowy = RandomForestClassifier(n_estimators = 555, bootstrap = bool("False"), min_samples_split= 0.06123690495575964,
                                            max_features =  0.6767722944967449)
model = Las_losowy.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(auc)

0.9063349269044894
